<a href="https://colab.research.google.com/github/titsitits/Python_Data_Science/blob/master/7_Machine_Learning_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mickaël Tits
CETIC
mickael.tits@cetic.be

# Introduction au machine learning

Un sous-domaine important de l'analyse de données est le machine learning (apprentissage automatique). Le machine learning regroupe un ensemble de techniques permettant, à partir des données, d'apprendre automatiquement différents types de relations entre les variables.

On distingue généralement:
* Le machine learning **supervisé**, qui permet d'apprendre des relations entre les données et des labels. Plus spécifiquement, on entraîne généralement un algorithme à prédire les labels (aussi appelés **variables dépendantes**, ou **targets**) un ensemble de variables (appelées **variables prédictives**, ou **features**). Un modèle consiste en un algorithme doté de **paramètres** et qui, à partir d'opérations entre les paramètres et les variables prédictives, prédit un nouveau label. L'apprentissage consiste alors à modifier les paramètres du modèle de manière à ce que les labels prédits soient les plus proches des labels réels. Autrement dit, on **minimise l'erreur moyenne des prédictions**. Cette minimisation de l'erreur se fait généralement par de manière itérative, par **descente de gradient**.

  Il existe deux applications principales de l'apprentissage supervisé:
  
  * La **classification**: on a un ensemble de données, par exemple des images, à partir desquelles on aimerait prédire une classe: le **label** de chaque image est alors une variable catégorielle, par exemple "chat" ou "chien". Les **variables prédictives** permettant d'entraîner l'algorithme sont par exemple les pixels des images, ou des relations entre ces pixels.
  
  * La **régression**: le **label** à prédire à partir des données est une valeur continue: par exemple le prix d'une maison, qu'on peut prédire à partir de différentes caractéristiques, ou l'âge d'une personne à prédire à partir d'une image.

* Le machine learning **non-supervisé**, qui permet d'apprendre des relations entre différentes données, telles que les similitudes entre différentes variables, ou différentes données (e.g.: Clustering, Réduction de dimensions).


Chargez d'abord le dataframe préparé lors du chapitre précédent (relancer le notebook précédent si nécessaire). 

**Remarque**: Pour que celui-ci soit présent dans la machine virtuelle en cours, il faut avoir exécuté au préalable les deux notebooks précédents. Vous rencontrerez donc une erreur si ce n'est pas le cas.

In [0]:
import pandas as pd

df = pd.read_csv("houses_features.csv", index_col = 0)
dfbxl = df[df.city == "Bruxelles"]
df

# Prédiction du prix d'une maison

## Préparation du dataset

In [0]:
#L'adresse n'a pas d'intérêt (on a déjà extrait la ville, qui est une catégorie intéressante)
dataset = df.drop("address",axis=1)
#dataset = dfbxl.drop("address",axis=1)


De nombreux algorithmes de machine learning nécessitent l'utilisation exclusive de **variables prédictives numériques**. En effet, ces algorithmes se basent entièrement sur des opérations algébriques à partir de ces variables pour prédire la variable dépendante.
Afin de pouvoir utiliser l'information contenue dans les variables catégorielles, il existe différentes techniques pour les transformer en variables continues:

* **One-hot encoding** (dummy variable extraction): Pour chaque catégorie possible d'une variable catégorielle, on crée une variable binaire, indiquant si l'échantillon est de cette catégorie ou non. Par exemple, la variable "`Ville`" ["Bruxelles" ou "Namur"] peut être simplement transformée en une variable binaire "`is_bruxelles`": "Bruxelles" => 1, "Namur" => 0

  Si `Ville` contient trois possibilités (e.g., ["Bruxelles" ou "Namur" ou "Fleurus"]), elle nécessitera la création de variables binaires, e.g.: `is_bruxelles` et `is_namur`. Pour chaque observation, une seule des deux variables ne peut être à 1 (puisque la variable `Ville` ne peut être qu'une ville à la fois). Si les deux sont fausses, c'est que la ville est Fleurus.
  
  De manière générale, une variable à n catégories nécessitera la création de n-1 variables binaires. On appelle généralement ces variables **dummy**.
  
* **Echelle** : certaines variables sont ordinale, comme `quality` [`bien`, `moyen`, `mauvais`]. Celle-ci peut être directement transformée un variable numérique: [bien => 0, moyen = >1, mauvais => 2]. Si ce n'est pas le cas, l'interprétation sémantique de la variable, et sa combinaison avec une autre variable continue permet de définir une échelle. Par exemple, on peut extraire certaines caractéristiques numériques d'une ville à partir du prix moyen des maisons, ou du nombre d'habitants: ["Bruxelles" => 300000, "Namur" => 250000, "Fleurus" => 200000].




In [0]:
#One-hot encoding des variables catégorielles
dataset = pd.get_dummies(dataset, drop_first=True)
dataset

In [0]:
dataset.corr()

## Une framework Python de référence pour le machine learing: Scikit-learn

Une librairie Python incontournable pour le machine learning est la librairie Scikit-learn: https://scikit-learn.org/stable/ .

Elle inclut un geand nombre d'algorithmes de machine learning supervisé, non-supervisé, de préprocessing de caractéristiques, et de sélection de modèles de machine learning. L'API est assez simple, et est basée sur les librairies vues précédemment (Numpy, Scipy, Matplotlib).

La librairie contient de nombreuses classes permettant d'instancier des modèles de machine learning, que l'on peut ensuite utiliser pour: 
1. Entraîner sur un ensemble de **données d'entraînement (ou training set)**.

2. Chaque modèle contient généralement différents paramètres qui peuvent être adaptés aux données utilisées, afin d'améliorer les performances de l'algorithme. Pour vérifier les performances du modèles selon différents paramètres, on teste le modèle sur un set de **données de validation (ou validation set)**. On parle généralement de processus de **cross-validation**.

3. Afin de vérifier que le modèle obtenu est efficace, on peut ensuite le tester avec de nouvelles données (autres que celles utilisées durant l'entraînement et la cross-validation), les **données de test (ou test set)**.

### Régression linéaire

Etant donné le peu de données de l'exemple, on utilisera un modèle simple (régression linéaire), avec un nombre restreint de caractéristiques (surface, rooms, et éventuellement la ville). On ne contentera d'un training set et d'un test set (pas de validation ici: on gardera les paramètres par défaut du modèle).

In [0]:
from sklearn.linear_model import LinearRegression

#On crée un objet de la classe LinearRegression
regressor = LinearRegression()


In [0]:
trainsize = 6

trainset = dataset.iloc[:trainsize]
testset = dataset.iloc[trainsize:]

features = ["surface","rooms"]
#Pour tester un autre set de features, décommenter la ligne suivante
features = ["surface","rooms","city_Fleurus","city_Namur"]

Xtrain, ytrain, Xtest, ytest = trainset[features], trainset["price"], testset[features], testset["price"]

#### Entraînement

In [0]:
regressor.fit(Xtrain, ytrain)

####Prédictions

In [0]:
trainpred = regressor.predict(Xtrain)
testpred = regressor.predict(Xtest)
print(testpred[0], ytest.values[0])

In [0]:
regressor.coef_

#### Evaluation des prédictions

In [0]:
from sklearn.metrics import mean_absolute_error

mae_train = mean_absolute_error(trainpred,ytrain)
mae_test = mean_absolute_error(testpred,ytest)

mae_train, mae_test

In [0]:
trainpred

In [0]:
testpred

In [0]:
ytest

In [0]:
from matplotlib import pyplot as plt

plt.scatter(ytrain,trainpred)
plt.scatter(ytest,testpred)


plt.legend(["train","test"])
plt.xlabel("Labels")
plt.ylabel("Prédictions")

Le modèle semble indiquer qu'une des maisons du testset est sous-évaluée: le modèle estime sa valeur plus haute que le prix auquel elle est vendue.

In [0]:
output = df.iloc[trainsize:]
output = output.assign(predictions = testpred)
output["delta"] = output["price"] - output["predictions"]
output

La maison à la Rue de Fer 29, Namur	serait donc fortement sous-évaluée. On remarque en l'occurrence qu'elle a un prix par pièce particulièrement faible.

In [0]:
df

# Importance des variables prédictives

On peut évaluer l'impact des variables prédictives sur la prédiction à partir de leur coefficient dans la régression linéaire. Néanmoins, pour que les coefficients soient comparables, il faut au préalable mettre à la même échelle les variable prédictives.

In [0]:
from sklearn.preprocessing import StandardScaler

#Scaling
scaler = StandardScaler()
Xtrain_scaled = scaler.fit_transform(Xtrain)
Xtest_scaled = scaler.transform(Xtest)

#Entraînement
regressor.fit(Xtrain_scaled, ytrain)

#Prédictions
trainpred = regressor.predict(Xtrain_scaled)
testpred = regressor.predict(Xtest_scaled)

(features,regressor.coef_)

Le coefficient le plus grand en valeur absolue est le deuxième, et semble indiquer que c'est le nombre de pièces qui influence le plus le prix.

### La descente de gradient

In [0]:
from sklearn.linear_model import SGDRegressor

#On crée un objet de la classe LinearRegression
regressor = SGDRegressor(loss = "squared_loss", learning_rate = "constant", eta0 = 0.01, max_iter = 300)
regressor.fit(Xtrain_scaled, ytrain)
print(regressor.predict(Xtest_scaled)[0], ytest.values[0])

## Leave-one-out validation
Lorsque l'ensemble de données est petit (comme dans cet exemple), il est intéressant d'utiliser une technique de tournante sur les données de test pour vérifier la qualité de l'algorithme choisi. Sur l'ensemble des données, on retient itérativement un sous-ensemble différent pour les données de test et on entraîne sur le reste. On peut parcourir itérativement toutes les données, pour ainsi tester au final l'algorithme sur l'ensemble des données.

In [0]:
from sklearn.metrics import mean_squared_error
import numpy as np

testpredictions = []

input_features = features


for i in dataset.index:
  trainset = dataset[dataset.index!=i]
  testset = dataset.loc[i].to_frame().transpose()
  Xtrain, ytrain, Xtest, ytest = trainset[input_features], trainset["price"], testset[input_features], testset["price"]
  
  scaler = StandardScaler()
  Xtrain_scaled = scaler.fit_transform(Xtrain)
  Xtest_scaled = scaler.transform(Xtest)

  regressor.fit(Xtrain_scaled, ytrain)
  
  trainpred = regressor.predict(Xtrain_scaled)
  testpred = regressor.predict(Xtest_scaled)
  
  print(i, " - training mean error:", np.mean(np.abs(trainpred - ytrain)), "\n\t test mean error:", np.mean(np.abs(testpred - ytest)), "\n")
        
  testpredictions.append(testpred[0])
  

In [0]:
testpredictions

In [0]:
mean_absolute_error(dataset["price"],testpredictions)

Si les performances sur le test set sont bien plus mauvaises que sur les données d'entraînement, il est fort probable qu'on soit dans une situation d'**overfitting**: le modèle a appris "par coeur" ce qu'il fallait prédire pour chaque donnée du training set. Il ne se généralise donc pas bien sur de nouvelles données. Afin d'éviter ce problème, on peut soit:

* diminuer la complexité (le nombre de degrés de liberté, i.e. de paramètres) du modèle, soit en diminuant le nombre de caractéristique, soit en utilisant un algorithme plus simple.
* entraîner le modèle sur plus de données.

Si à l'inverse, les performances obtenues lors de l'entraînement sont faibles, on est en situation d'**underfitting**. Il faut alors au contraire augmenter la complexité au modèle et extraire des caractéristiques de meilleure qualité (ayant une relation pus forte avec le label à prédire).

Le dataset minimaliste utilisé ici a permis de présenter très simplement certains concepts de base du machine learning. Néanmoins, en pratique, l'utilisation du machine learning nécessite généralement de beaucoup plus grands ensemble de données, permettant notamment de développer des modèles plus complexes qu'une régression linéaire. Dans le Chapitre suivant, nous utiliserons un dataset réel et plus pertinent pour explorer quelques concepts fondamentaux du machine learning.
